In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.layers import*
from keras.models import*
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!unzip alz.zip

Archive:  alz.zip
  inflating: oasis_cross-sectional.csv  
  inflating: oasis_longitudinal.csv  


In [ ]:
df1 = pd.read_csv('/content/oasis_longitudinal.csv')
df1 = df1.fillna(method='ffill')
df2 = pd.read_csv('/content/oasis_cross-sectional.csv')
df2 = df2.fillna(method='ffill')

In [ ]:
df1.drop(['MRI ID'],axis=1,inplace=True)
df1.drop(['Visit'],axis=1,inplace=True)

In [ ]:
df1['CDR'].replace(to_replace=0.0, value='A',inplace=True)
df1['CDR'].replace(to_replace=0.5, value='B',inplace=True)
df1['CDR'].replace(to_replace=1.0, value='C',inplace=True)
df1['CDR'].replace(to_replace=2.0, value='D',inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
for x in df1.columns:
  f = LabelEncoder()
  df1[x] = f.fit_transform(df1[x])

In [ ]:
df1.head()

,Subject ID,Group,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,0,2,0,1,0,27,5,1,14,0,284,35,1
1,0,2,15,1,0,28,5,1,17,0,285,20,0
2,1,1,0,1,0,15,3,1,10,1,231,73,52
3,1,1,32,1,0,16,3,1,15,1,254,51,32
4,1,1,185,1,0,20,3,1,9,1,238,40,46


In [ ]:
for i in df2.columns:
  df2[i] = LabelEncoder().fit_transform(df2[i])

In [ ]:
df2.head()

,ID,M/F,Hand,Age,Educ,SES,MMSE,CDR,eTIV,nWBV,ASF,Delay
0,0,0,0,51,1,2,15,0,67,62,214,212
1,1,0,0,33,3,0,15,0,2,115,279,290
2,2,0,0,50,3,2,13,1,140,34,148,289
3,3,1,0,10,3,2,13,1,228,109,73,288
4,4,1,0,0,3,2,13,1,284,151,24,287


In [ ]:
data = pd.concat([df1,df2])
data.fillna(method='ffill')

In [ ]:
data.head()

,Subject ID,Group,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF,ID,Educ,Delay
0,0.0,2.0,0.0,1,0,27,5.0,1,14,0,284,35,1,NaN,NaN,NaN
1,0.0,2.0,15.0,1,0,28,5.0,1,17,0,285,20,0,NaN,NaN,NaN
2,1.0,1.0,0.0,1,0,15,3.0,1,10,1,231,73,52,NaN,NaN,NaN
3,1.0,1.0,32.0,1,0,16,3.0,1,15,1,254,51,32,NaN,NaN,NaN
4,1.0,1.0,185.0,1,0,20,3.0,1,9,1,238,40,46,NaN,NaN,NaN


In [ ]:
data_mat = data.corr()
sns.heatmap(data_mat,annot=True)

In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(df1,test_size=0.20,random_state=1)
xtrain = train[['Group','MR Delay','M/F','Hand','Age','EDUC','SES','MMSE','eTIV','nWBV','ASF']].fillna(method='ffill')
ytrain = train.CDR
xtest = test[['Group','MR Delay','M/F','Hand','Age','EDUC','SES','MMSE','eTIV','nWBV','ASF']].fillna(method='ffill')
ytest = test.CDR 

In [ ]:
from sklearn.preprocessing import StandardScaler
Xtrain = xtrain.astype('float32')
Xtest = xtest.astype('float32')
Xtrain1 = StandardScaler().fit_transform(Xtrain)
Xtest1 = StandardScaler().fit_transform(Xtest)
ytrain1 = np.ravel(ytrain)
ytest1 = np.ravel(ytest)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression().fit(Xtrain1,ytrain1)
predict = classifier.predict(Xtest1)
print('Train score',classifier.score(Xtrain1,ytrain1))
print('Test score',classifier.score(Xtest1,ytest1))

Train score 0.8221476510067114
Test score 0.7866666666666666


In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=5).fit(Xtrain1,ytrain1)
predict = tree.predict(Xtest1)
print('Train score',classifier.score(Xtrain1,ytrain1))
print('Test score',classifier.score(Xtest1,ytest1))

Train score 0.8221476510067114
Test score 0.7866666666666666


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5).fit(Xtrain1,ytrain1)
predict = knn.predict(Xtest1)
print('Train score',classifier.score(Xtrain1,ytrain1))
print('Test score',classifier.score(Xtest1,ytest1))

Train score 0.8221476510067114
Test score 0.7866666666666666


In [ ]:
ytrain1.shape

(298,)